In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import datetime
from text_cnn import TextCNN
from tensorflow.contrib import learn
import re
import time
import MeCab
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from collections import OrderedDict
from gensim.models import word2vec
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
All_df = pd.read_pickle("data/all_mecab.pickle")
All_df

,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,21,21,31,31,40,40,50,50
1,"[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,21,21,31,31,40,40,50,50
2,"[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,21,21,31,31,40,40,50,50
3,"[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...",119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,21,21,31,31,40,40,52,52
4,"[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]",163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,22,22,32,32,41,40,50,50
5,"[使い, にくい, です, 笑, EOS]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...",194,-1,1234568,まこぴす,使いにくいです(笑),2,2,22,22,32,32,41,40,50,50
6,"[同意, です, EOS]","[使い, にくい, です, 笑, EOS]",302,-1,1234568,哲,同意です,1,1,22,22,32,32,42,42,50,50
7,"[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...","[同意, です, EOS]",309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,23,23,32,32,41,41,50,50
8,"[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...",385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,25,25,32,32,41,41,50,50
9,"[自分, の, 意見, です, ！, 笑, EOS]","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...",426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,21,22,31,32,40,41,50,50


In [3]:
sen = All_df['sen'].values
sen_pre = All_df['sen_pre'].values

# 単語辞書の作成
wd_set = Counter([x for s in (sen + sen_pre) for x in s])
wd_ary = np.array(list(wd_set.keys()))
wd_cnt = np.array(list(wd_set.values()))

# 出現頻度順にソート
wd_ary = wd_ary[np.argsort(wd_cnt)[::-1]]
wd_cnt.sort()
wd_cnt = wd_cnt[::-1]

# 単語ID辞書の作成
wd_to_id = {wd: i for i, wd in enumerate(wd_ary)}

# Top10の単語を出力
for i in range(10):
    print("単語:",
          list(wd_ary)[i], "\t出現数:",
          list(wd_cnt)[i], "\tID:", wd_to_id[list(wd_ary)[i]])

# 出現数CUT_OFF以下の単語のIDを統一
CUT_OFF = 2
print("words kinds:", len(wd_cnt), "words>=" + str(CUT_OFF) + ":",
      np.sum(wd_cnt >= CUT_OFF))
other_id = np.sum(wd_cnt >= CUT_OFF)
wd_to_id.update({wd: other_id for wd in wd_ary[wd_cnt < CUT_OFF]})
id_to_wd = {wd_to_id[wd]: wd for wd in wd_to_id.keys()}

# senとsen_preの単語をIDに変換
sen_id = []
sen_pre_id = []
for s, s_pre in zip(sen, sen_pre):
    sen_id.append([str(wd_to_id[wd]) for wd in s])
    sen_pre_id.append([str(wd_to_id[wd]) for wd in s_pre])

# 新し列としてAll_dfに追加
All_df.insert(loc=0, column='sen_id', value=sen_id)
All_df.insert(loc=1, column='sen_pre_id', value=sen_pre_id)

単語: EOS 	出現数: 29580 	ID: 0
単語: の 	出現数: 10567 	ID: 1
単語: て 	出現数: 7408 	ID: 2
単語: です 	出現数: 7390 	ID: 3
単語: ます 	出現数: 7363 	ID: 4
単語: か 	出現数: 7285 	ID: 5
単語: 、 	出現数: 6959 	ID: 6
単語: に 	出現数: 6914 	ID: 7
単語: が 	出現数: 6893 	ID: 8
単語: は 	出現数: 6793 	ID: 9
words kinds: 6961 words>=2: 6649


In [4]:
All_df

,sen_id,sen_pre_id,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[39, 35, 12, 4, 18, 0]",[0],"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,21,21,31,31,40,40,50,50
1,"[39, 35, 12, 4, 0]","[39, 35, 12, 4, 18, 0]","[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,21,21,31,31,40,40,50,50
2,"[413, 19, 1, 8, 2290, 3, 0, 39, 35, 12, 4, 18, 0]","[39, 35, 12, 4, 0]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,21,21,31,31,40,40,50,50
3,"[1813, 45, 142, 298, 56, 54, 27, 18, 0]","[413, 19, 1, 8, 2290, 3, 0, 39, 35, 12, 4, 18, 0]","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...",119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,21,21,31,31,40,40,52,52
4,"[142, 298, 56, 54, 27, 0, 663, 9, 2280, 3, 18, 0]","[1813, 45, 142, 298, 56, 54, 27, 18, 0]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]",163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,22,22,32,32,41,40,50,50
5,"[196, 917, 3, 112, 0]","[142, 298, 56, 54, 27, 0, 663, 9, 2280, 3, 18, 0]","[使い, にくい, です, 笑, EOS]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...",194,-1,1234568,まこぴす,使いにくいです(笑),2,2,22,22,32,32,41,40,50,50
6,"[1768, 3, 0]","[196, 917, 3, 112, 0]","[同意, です, EOS]","[使い, にくい, です, 笑, EOS]",302,-1,1234568,哲,同意です,1,1,22,22,32,32,42,42,50,50
7,"[1658, 1387, 6398, 738, 12, 2, 56, 10, 1, 3, 8...","[1768, 3, 0]","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...","[同意, です, EOS]",309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,23,23,32,32,41,41,50,50
8,"[44, 425, 118, 168, 1826, 1, 181, 9, 5314, 2, ...","[1658, 1387, 6398, 738, 12, 2, 56, 10, 1, 3, 8...","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...",385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,25,25,32,32,41,41,50,50
9,"[68, 1, 132, 3, 18, 112, 0]","[44, 425, 118, 168, 1826, 1, 181, 9, 5314, 2, ...","[自分, の, 意見, です, ！, 笑, EOS]","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...",426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,21,22,31,32,40,41,50,50


In [5]:
# データの整理（一致、重複）
print("全データ：", All_df.shape)
drop_All_df = All_df.drop_duplicates(subset=['body', 'name']).reset_index(
    drop=True)
print("重複投稿を排除したデータ：", drop_All_df.shape)

# 16types-------------------------------------
t16_All_df = drop_All_df
print("16types有効データ(重複削除)：", t16_All_df.shape)
valid = t16_All_df.shape[0]
t16_All_df = t16_All_df[t16_All_df['16types_a'] == t16_All_df[
    '16types_b']].reset_index(drop=True)
print("16types一致データ(重複削除)：", t16_All_df.shape)
print("16types一致率：", valid, "/", t16_All_df.shape[0], "=",
      t16_All_df.shape[0] / valid)
print(Counter(t16_All_df['16types_a']), "\n")

全データ： (12012, 19)
重複投稿を排除したデータ： (11357, 19)
16types有効データ(重複削除)： (11357, 19)
16types一致データ(重複削除)： (7926, 19)
16types一致率： 11357 / 7926 = 0.6978955710134719
Counter({1: 1867, 2: 1325, 5: 1046, 3: 900, 4: 885, 6: 779, 8: 236, 7: 227, 14: 179, 9: 141, 11: 108, 15: 102, 12: 63, 13: 30, 0: 21, 10: 17}) 



In [6]:
x_tx = t16_All_df['sen_id'].values

x_text = [" ".join(x) for x in x_tx]


y_lb = t16_All_df['16types_a'].values
y_lb = np.array(y_lb,dtype=np.int32)
y = np.eye(16)[y_lb]

print(len(x_text))
print(type(x_text))
print(x_text[0])

print(y.shape)
print(type(y))
print(type(y[0]))
print(y[0])

7926
<class 'list'>
39 35 12 4 18 0
(7926, 16)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
max_document_length = max([len(x.split(" ")) for x in x_text])
print(max_document_length,"\n")
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))
print(len(vocab_processor.vocabulary_),"\n")
print(x.shape)
print(x[0],"\n")

np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

dev_sample_percentage = 0.1
dev_sample_index = -1 * int(dev_sample_percentage * float(len(y)))
print(dev_sample_index,"\n")
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

print(x_train.shape)
print(x_train[0],"\n")
print(x_dev.shape)
print(x_dev[0],"\n")
print(y_train.shape)
print(y_train[0],"\n")
print(y_dev.shape)
print(y_dev[0],"\n")

292 

5495 

(7926, 292)
[1 2 3 4 5 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 

-792 

(7134, 292)
[776 777 776 776 777 776   6   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [8]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [9]:
session_conf = tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=False)

sess = tf.Session(config=session_conf)

embedding_dim = 128
filter_sizes = "3,4,5"
num_filters = 128
l2_reg_lambda = 0.0
dropout_keep_prob = 0.5
batch_size = 64
num_epochs = 200
evaluate_every = 100
checkpoint_every = 100

with sess.as_default():
    cnn = TextCNN(
        sequence_length=x_train.shape[1],
        num_classes=y_train.shape[1],
        vocab_size=len(vocab_processor.vocabulary_),
        embedding_size=embedding_dim,
        filter_sizes=list(map(int, filter_sizes.split(","))),
        num_filters=num_filters,
        l2_reg_lambda=l2_reg_lambda)

    global_step = tf.Variable(0, trainable=False)
    optimizer = tf.train.AdamOptimizer(1e-3)
    grads_and_vars = optimizer.compute_gradients(cnn.loss)
    train_op = optimizer.apply_gradients(
        grads_and_vars, global_step=global_step)

    sess.run(tf.global_variables_initializer())

    def train_step(x_batch, y_batch):
        feed_dict = {
            cnn.input_x: x_batch,
            cnn.input_y: y_batch,
            cnn.dropout_keep_prob: dropout_keep_prob
        }
        _, step, loss, accuracy = sess.run(
            [train_op, global_step, cnn.loss, cnn.accuracy], feed_dict)
        print("step:{}\t loss:{:g}\t acc:{:g}".format(step, loss,
                                                  accuracy))

    def dev_step(x_batch, y_batch):
        feed_dict = {
            cnn.input_x: x_batch,
            cnn.input_y: y_batch,
            cnn.dropout_keep_prob: 1.0
        }
        step, loss, accuracy = sess.run([global_step, cnn.loss, cnn.accuracy],
                                        feed_dict)
        print("step:{}\t loss:{:g}\t acc:{:g}".format(step, loss, accuracy))
        
    batches = batch_iter(list(zip(x_train, y_train)), batch_size, num_epochs)
    
    
    for batch in batches:
        x_batch, y_batch = zip(*batch)
        train_step(x_batch, y_batch)
        current_step = tf.train.global_step(sess, global_step)
        if current_step % evaluate_every == 0:
            print("\nEvaluation:")
            dev_step(x_dev, y_dev)
            print("")


aaa: <class 'tensorflow.python.framework.ops.Tensor'>
aaa: <class 'tensorflow.python.framework.ops.Tensor'>
step:1	 loss:7.6062	 acc:0
step:2	 loss:6.56218	 acc:0.046875
step:3	 loss:5.53489	 acc:0.109375
step:4	 loss:4.91898	 acc:0.15625
step:5	 loss:4.19946	 acc:0.21875
step:6	 loss:3.65393	 acc:0.28125
step:7	 loss:4.28478	 acc:0.1875
step:8	 loss:4.71625	 acc:0.1875
step:9	 loss:4.12327	 acc:0.21875
step:10	 loss:4.03212	 acc:0.265625
step:11	 loss:3.35195	 acc:0.296875
step:12	 loss:4.07199	 acc:0.3125
step:13	 loss:4.09413	 acc:0.28125
step:14	 loss:4.18313	 acc:0.203125
step:15	 loss:3.0392	 acc:0.3125
step:16	 loss:3.10065	 acc:0.265625
step:17	 loss:3.72181	 acc:0.203125
step:18	 loss:3.61103	 acc:0.390625
step:19	 loss:3.33125	 acc:0.328125
step:20	 loss:4.1113	 acc:0.25
step:21	 loss:3.63656	 acc:0.265625
step:22	 loss:4.50147	 acc:0.21875
step:23	 loss:3.76256	 acc:0.3125
step:24	 loss:2.91663	 acc:0.296875
step:25	 loss:3.20291	 acc:0.28125
step:26	 loss:4.0272	 acc:0.2031

step:232	 loss:1.3483	 acc:0.546875
step:233	 loss:1.47917	 acc:0.53125
step:234	 loss:1.75655	 acc:0.515625
step:235	 loss:1.50187	 acc:0.578125
step:236	 loss:1.95645	 acc:0.546875
step:237	 loss:1.73197	 acc:0.546875
step:238	 loss:1.3392	 acc:0.609375
step:239	 loss:1.88691	 acc:0.546875
step:240	 loss:1.41357	 acc:0.53125
step:241	 loss:1.72849	 acc:0.53125
step:242	 loss:1.62669	 acc:0.59375
step:243	 loss:1.47404	 acc:0.625
step:244	 loss:1.82533	 acc:0.515625
step:245	 loss:1.55068	 acc:0.515625
step:246	 loss:1.56988	 acc:0.578125
step:247	 loss:1.40212	 acc:0.625
step:248	 loss:2.21295	 acc:0.359375
step:249	 loss:1.3182	 acc:0.5625
step:250	 loss:1.82818	 acc:0.5
step:251	 loss:1.63857	 acc:0.5
step:252	 loss:0.8835	 acc:0.6875
step:253	 loss:1.6129	 acc:0.5625
step:254	 loss:1.3247	 acc:0.578125
step:255	 loss:1.86272	 acc:0.515625
step:256	 loss:1.99783	 acc:0.53125
step:257	 loss:1.69038	 acc:0.59375
step:258	 loss:1.71548	 acc:0.5
step:259	 loss:1.47036	 acc:0.59375
step

KeyboardInterrupt: 